In this notebook, we will be attempting to use transfer learning to predict our labels.

In [30]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16

from pathlib import Path
import imghdr

### Pre-process data

In [31]:
#  the data
data_dir = "./BBTrD/000000001111"
image_extensions = [".png", ".jpg", ".jpeg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

### Load Images and Labels

In [32]:
def load_labels():
    txt_file_path = "labels.txt"
    labels = []
    with open(txt_file_path, 'r') as f:
        for line in f:
            labels.append(line.strip())
    return labels

In [33]:
game_states = load_labels()
game_states = sorted(game_states)
game_states

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

In [34]:
image_height = 128
image_width = 128
batch_size = 81

def load_data():
    # Load Train Data
    train_images = tf.keras.utils.image_dataset_from_directory("./BBTrD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)

    # Load Test Data
    test_images =tf.keras.utils.image_dataset_from_directory("./BBTeD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)
    return train_images, test_images

In [47]:
train_images, test_images = load_data()

Found 81 files belonging to 81 classes.
Found 34 files belonging to 34 classes.


In [48]:
train_images.class_names

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

## Resize & Rescale Images

In [36]:
resize_and_rescale = tf.keras.Sequential([
    keras.layers.Rescaling(1./255)
])

train_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))
test_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))

## Data Augmentation

In [53]:
data_flip = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical")
])

data_rotation = tf.keras.Sequential([
  keras.layers.RandomRotation(0.2)
])

flipped_images = train_images.map(lambda x, y: (data_flip(x), y))
rotated_images = train_images.map(lambda x, y: (data_rotation(x), y))

train_images = train_images.concatenate(flipped_images)
train_images = train_images.concatenate(rotated_images)

### Load Pre-trained VGG model

In [54]:
#load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [55]:
num_classes = 81
num_epochs = 10
# we will be freezing the base model's layers in order to keep the weights in the optimal condition they were trained in
for layer in vgg_conv.layers:
    layer.trainable = False

# new classifier model, which consists of the old model's layers plus some new ones
extra_layers = vgg_conv.output
extra_layers = keras.layers.Flatten()(extra_layers)
extra_layers = keras.layers.Dense(1024, activation='relu')(extra_layers)
predictions = keras.layers.Dense(num_classes, activation='softmax')(extra_layers)

# create the new model
new_model = keras.models.Model(inputs=vgg_conv.input, outputs=predictions)

# compile the model
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
new_model.fit(train_images, epochs=num_epochs, validation_data=test_images)

# print(train_images)





Epoch 1/10
3/3 [==============================] - 57s 16s/step - loss: 57.3983 - accuracy: 0.0247 - val_loss: 98.3358 - val_accuracy: 0.0294
Epoch 2/10
3/3 [==============================] - 60s 18s/step - loss: 68.2289 - accuracy: 0.1440 - val_loss: 105.7269 - val_accuracy: 0.0294
Epoch 3/10
3/3 [==============================] - 66s 18s/step - loss: 71.1418 - accuracy: 0.2716 - val_loss: 101.8980 - val_accuracy: 0.0588
Epoch 4/10
3/3 [==============================] - 65s 18s/step - loss: 72.2350 - accuracy: 0.3004 - val_loss: 111.2792 - val_accuracy: 0.0294
Epoch 5/10
3/3 [==============================] - 67s 19s/step - loss: 60.9364 - accuracy: 0.4198 - val_loss: 108.5960 - val_accuracy: 0.0294
Epoch 6/10
3/3 [==============================] - 71s 23s/step - loss: 53.4034 - accuracy: 0.4527 - val_loss: 96.2907 - val_accuracy: 0.0882
Epoch 7/10
3/3 [==============================] - 62s 17s/step - loss: 42.3240 - accuracy: 0.5350 - val_loss: 87.3350 - val_accuracy: 0.0294
Epoch 8/1

In [51]:
new_model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0  